# Method / Attribute

* Control object's behaviors
* Data / Behavior

# Object's three specific things
* **Identity**
    * Unique - Used to compare objects to each other without looking at any details
    * Address in memory
    * Not Shared
    * Can't be modified at runtime > Can be reused once it's destroyed


In [ ]:
class A:
    def __init__(self):
        print("A: Init")
a = A
aa = A
print(id(a))
print(id(aa))

* **Type**
    * Defined by its class and any base classes
    * Shared


* **Value**
    * Provided by a namespace dictionary that's specific to a given object 
    * Designed to work with the type 

# Namespace Dictionary

Object's namespace
* Implemented as a dictionary that's created 
* Used to store values for all the attributes on the object
* **Can be accesses and modified at run-time**
 * Can be replaced with another dictionary
* *\__dict__*


## Example: Borg Patttern

A large number of instances to share a single namespace
* The identity remains distinct

**`__init__()`**
* All subclasses would need to make sure they use **`super()`** in order to call the initialization prodedures from the Bog classes
    * Use **`super()`** before doing any attribute assignments of their own
*

In [ ]:
class Borg:
    _namespace = {}
    def __init__(self):
        self.__dict__ = Borg._namespace

a = Borg()
b = Borg()

In [ ]:
hasattr(a, 'attribute')

In [ ]:
b.attribute = 'value'
hasattr(a, 'attribute')

In [ ]:
a.attribute

In [ ]:
Borg._namespace

**Mixin**

In [ ]:
class Base:
    def __init__(self):
        print('Base')

class Borg:
    _namespace = {}
    def __init__(self, *args, **kwargs):
        self.__dict__ = Borg._namespace
        print('Borg')

class Testing(Borg, Base):
    pass
print(Testing())
class Testing(Base, Borg):
    pass
print(Testing())

[**`__new()__`**](https://docs.python.org/3/reference/datamodel.html#object.__new__)
* `__new()__` is less commonly implemented
    * The odds of running into conflicting implementations are much smaller
* The object must be created along the way, usually by calling `__new__()` on the base object.

In [ ]:
# For all classes that inherit from Borg 
class Base:
    def __init__(self):
        print('Base')
class Borg:
    _namespace = {}
    def __new__(cls, *args, **kwargs):
        print('Borg')
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._namespace
        return obj
class TestingOne(Borg, Base):
    pass

print("# First #")
print(TestingOne())
class TestingTwo(Base, Borg):
    pass

print("# Second #")
print(TestingTwo())
a = TestingOne()
Borg
Base
b = TestingTwo()
Borg
Base
a.attribute = 'value'
print("B Attribute:" + b.attribute)


**Apply Borg only to those classes where it is applied**
* `__new()__` must **create a new dictionary for each new class it encounters**, assigning it to a value in the existing namespace dictionary

In [ ]:
class Borg:
    _namespace = {}
    def __new__(cls, *args, **kwargs):
        print("Borg")
        print(cls)
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._namespace.setdefault(cls, {})
        return obj
class TestOne(Borg):
    pass
class TestTwo(Borg):
    pass
a = TestOne()
b = TestOne()

In [ ]:
a.spam = 'eggs'
b.spam

In [ ]:
c = TestTwo()
c.spam

In [ ]:
c.spam = 'Burger'
d = TestTwo()
d.spam

In [ ]:
a.spam

In [ ]:
Borg._namespace

## Self-Caching Properties

**Acess to properties**
* Every access to the attribute perform a lookup in the namespace.
 * A value must have been created and stored previously
 * The attribute value can be a complex object
 * Example: **`ORM(Object-Relational Mapping)`** sitting between application code and a relational databases

**Caching function's return value**
* Supply the name of the attribute as an argument to **`cacheproperty()`** in addition to naming the function

In [ ]:
import functools

def cacheproperty(name):
    def decorator(func):
        @property
        @functools.wraps(func)
        def wrapper(self):
            if name not in self.__dict__:
                self.__dict__[name] = func(self) # Store function's return value
            return self.__dict__[name]
        return wrapper
    return decorator

In [ ]:
class Example:
    @cacheproperty('attr')
    def attr(self):
        print('Gettring the value!')
        return 42
    
e = Example()
e.attr

In [ ]:
e.attr

In [ ]:
def cachedproperty(func):
    values = {}
    @property
    @functools.wraps(func)
    def wrapper(self):
        if self not in values:
            values[self] = func(self)
        return values[self]
    return wrapper

class Example:
    @cacheproperty
    def attr(self):
        print('Gettring the value!')
        return 42
    
e = Example()
e.attr()

In [ ]:
e.attr

# Garbage Collection

Key actions for GC
* Identify an object as garbage(not used )
* Remove garbage from meory


Variables that are changed during a command session 
* Re-referenced if you re-declare variable with a previously used value during that session

In [ ]:
x=10
type(x)

In [ ]:
id(x)

In [ ]:
x="foobar"
type(x)

In [ ]:
id(x)

In [ ]:
x=10
id(x)

## Rererence Counting

**How to reference an object.**
* Assign an object to any namespace
  > Increase reference count

**Createa reference**

In [1]:
a = [1,2,3]
b = {'example': a}
c = a

In [2]:
id(a)

4370093512

In [3]:
id(b['example'])

4370093512

In [5]:
id(c)

4370093512

In [12]:
import sys
sys.getrefcount(b['example'])

4

**Delete reference**

In [ ]:
del c
a = None
id(b['example'])

## Cyclical Reference

Only counting reference counts

In [35]:
# The dictionary and the list had one reference each other.
b = {'example': [1,2,3]} 
print(sys.getrefcount(b))
# Increase the dictionary's reference count by +1
b['example'].append(b)
print(sys.getrefcount(b))

2
3


**If so, What number of the dic's reference counts after del b?**
> decreased by -1

    Not garbage collected If reference counts are only considered.

**'Reference Cycle'**
* Any time a set of objects is referenced only by other objects in that set - and not from anywhere else in memory
* If one object is part of an orphaned reference cycle, any related objects are all also scheduled for deletion, so **which one should fire first?**

**How to handle a problem**
* Avoid having any objects with **`__del__()`** methods in any cyclical references
* Avoid having the objects appear in reference cycles.
* Provide a way that you can still detect them and have a change to clean them up on a regular basis.

In [54]:
import gc
class Example:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return 'Example %s' % self.value
    def __del__(self):
        print('Deleting %r' % self)
e = Example(1)

In [55]:
e

Example 1

In [51]:
del e

In [47]:
gc.collect()

0

In [57]:
gc.garbage

[]

## Weak Rererences

**Definition**
> Get a reference to the object without increasing its reference count

Use **`weakref`** module
* **`ref()`** class
 * Create a weak reference to whatever object is passed into it.

In [76]:
import weakref
class Example:
    pass

e = Example()
print(e)
ref = weakref.ref(e)
print(ref)
ref()
del e
print(ref)
print(ref())

<weakref at 0x10641f818; to 'Example' at 0x105ce7048>
<weakref at 0x10641f818; dead>
None


In [82]:
import weakref
class Example:
    pass

# Instantiating the object as part of the call ref(),
# the reference is inside of ref()
ref = weakref.ref(Example())
print(ref)
print(ref())

<weakref at 0x105cea188; dead>
None


Assignments in the function take place in that namespace, so once it's destroybed, **any objects assigned are destroyed as well unless they have references stored elsewhere**

In [84]:
def example():
    e = Example()
    ref = weakref.ref(e)
    return ref

e = example()
print(e)
print(e())

<weakref at 0x105cea688; dead>
None


In [89]:
e1 = Example()
print(sys.getrefcount(e1))
e2 = e1
print(sys.getrefcount(e1))
w = weakref.ref(e1)
print(sys.getrefcount(e1))

2
3
3


# Pickle

**Defintion**
* Convert a Python object into a persistent character stream that can be reloaded later
* Used for **serializing** and **deserializing**
* A sort of snapshot of the object at the time it was pickled

Objects
* Can be applied for list, dictionary..
* Can't be applifed for **function**, **classs**

Module
* **`pickle`** module
* dump() / dumps() methods

Protocol
* The pickle string always ocntains two bytes: **Protocol**
    * Tell which version of the pickling protocol to use

In [94]:
import pickle
print(pickle.dumps(1))
print(pickle.dumps(42))
print(pickle.dumps('42'))

b'\x80\x03K\x01.'
b'\x80\x03K*.'
b'\x80\x03X\x02\x00\x00\x0042q\x00.'


In [96]:
pickled = pickle.dumps(42)
print(pickled)
print(pickle.loads(pickled))

b'\x80\x03K*.'
42


# class Money:
    def __init__(self, amount, currency):
        self.amount = amount
        self.currency = currency
        self.conversion = {'USD':1, 'CAD':.95}
    def __str__(self):
        return '%.2f %s' % (self.amount, self.currency)
    def __repr__(self):
        return 'Money(%r, %r)' % (self.amount, self.currency)
    def in_currency(self, currency):
        ratio = self.conversion[currency] / self.conversion[self.currency]
        return Money(self.amount * ratio, currency)
us_dollar = Money(250, 'USD')
print(us_dollar)
print(us_dollar.in_currency('CAD'))
pickled = pickle.dumps(us_dollar)
print(pickled)
print(pickle.loads(pickled))

Required methods
* **`__getstate__(self)`**
* **`__setstate__(self, state)`**
    * With initialization code for pickled data

In [1]:
class Money:
    def __init__(self, amount, currency):
        self.amount = amount
        self.currency = currency
        #self.conversion = {'USD':1, 'CAD':.95}
        self.conversion = self.get_conversions()
    def __str__(self):
        return '%.2f %s' % (self.amount, self.currency)
    def __repr__(self):
        return 'Money(%r, %r)' % (self.amount, self.currency)
    def __getstate__(self):
        return self.amount, self.currency
    def __setstate__(self, state):
        self.amount = state[0]
        self.currency = state[1]
        self.conversion = self.get_conversions()
    def get_conversions(self):
        return {'USD':1, 'CAD':.95}
    def in_currency(self, currency):
        ratio = self.conversion[currency] / self.conversion[self.currency]
        return Money(self.amount * ratio, currency)
us_dollar = Money(250, 'USD')
print(us_dollar)
print(us_dollar.in_currency('CAD'))
pickled2 = pickle.dumps(us_dollar)
print(pickled2)
us_dollar_loads = pickle.loads(pickled2)
print(us_dollar_loads)
print(us_dollar_loads.in_currency('CAD'))

250.00 USD
237.50 CAD


NameError: name 'pickle' is not defined

# Copying

**Description**

* Changes made to an object are visible from every reference to that object
* Make changes to an object without those chagnes showing up elsewhere
: Need **Copying**

In [16]:
class B:
    def __init__(self, val):
        self.val = val
        
def checkf(param1):
    param1.val = 456
    print(param1.val)
    
a = B("val")
print(a.__dict__)
checkf(a)
print(a.__dict__)

{'val': 'val'}
456
{'val': 456}


**Slice: Copying the list**

In [14]:
a = [1,2,3]
b = a[:]
id(a)

4417888712

In [17]:
id(b)

4417808200

Dictionary: Use **`copy()`**

In [20]:
a = {1:2, 3:4}
b = a.copy()
id(a)

4417911880

In [22]:
id(b)

4417890184

## Shallow Copies

**`copy.copy()`**
* Create a new object with the same **type**, but with a new **identity** and a new - but identical - **value**
* Allow to pass in any object and get a shallow copy of it
* Can copy a wider variety of objects
* Can copy without needing to know anything about the objects themselves

In [23]:
import copy

In [24]:
class Example:
    def __init__(self, value):
        self.value = value

In [25]:
a = Example('spam')
b = copy.copy(a)
b.value = 'eggs'
a.value

'spam'

In [27]:
b.value

'eggs'

**References in the copied object**
* The value for the copied object may have a new namespace
* The namespaces all the same references
    * Only references get copied, not the objects themselves

In [31]:
a = {'a':[1,2,3], 'b':[4,5,6]}
b = a.copy()
a['a'].append(4)
b['b'].append(7)
a

{'a': [1, 2, 3, 4], 'b': [4, 5, 6, 7]}

In [33]:
b

{'a': [1, 2, 3, 4], 'b': [4, 5, 6, 7]}

**Example for shallow copy**: Keep an original list

In [35]:
def sorted(original_list, key=None):
    copied_list = copy.copy(original_list)
    copied_list.sort(key=key)
    return copied_list

a = [3,2,1]
b = sorted(a)
a

[3, 2, 1]

In [37]:
b

[1, 2, 3]

## Deep copies

**`copy.deepcopy()`** 
* Copy not only the original structure but also the objects that are referenced by it

In [39]:
import copy
original = [[1,2,3], [1,2,3]]
shallow_copy = copy.copy(original)
deep_copy = copy.deepcopy(original)
original[0].append(4)
print("Shallow")
print(shallow_copy)
print("Deep")
print(deep_copy)

Shallow
[[1, 2, 3, 4], [1, 2, 3]]
Deep
[[1, 2, 3], [1, 2, 3]]


In [41]:
import copy
b = {'example': [1,2,3]}
b['example'].append(b)
c = copy.deepcopy(b)
c

{'example': [1, 2, 3, {...}]}

In [46]:
b

[[1, 2, 3, 4], [1, 2, 3, 4]]

**The same object found in the structure**
* Only Copied once and referenced as many times as necessary

In [47]:
a = [1, 2, 3]
b = [a, a]
print(b)
b[0].append(4)
print(b)
c = copy.deepcopy(b)
print(c)
c[0].append(5)
print(c)

[[1, 2, 3], [1, 2, 3]]
[[1, 2, 3, 4], [1, 2, 3, 4]]
[[1, 2, 3, 4], [1, 2, 3, 4]]
[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]


**`__deepcopy()__`**
* Specify which values are pertinent to the copy
* Accept a second argument: The dictionary used to manage the identity of objects during copies